In [ ]:
import sys
import os

# Add the src directory to Python path so model.py can find ssn and net modules
sys.path.append(os.path.abspath('../src'))

from src.model import model
from src.model_outerweights import model_outerweights
from src.greedy_insertion import _sample_uniform_sphere_points
from src.training_logger import retrain

import numpy as np
from loguru import logger
import torch

In [ ]:
# load the data
path = '../data_result/raw_data/VDP_beta_0.1_grid_combined.npy'# Initialize the weights
data = np.load(path)
logger.info(f"Loaded data with shape: {data.shape}, dtype: {data.dtype}")

In [ ]:
# Initialize the parameter
power = 2.1
M = 50 # number greedy insertion selected
num_iterations = 10
loss_weights = (1.0, 1.0)
pruning_threshold = 1e-15

gamma = 5.0
alpha = 1e-5
lr_adam = 1e-5
regularization = (gamma, alpha) 
th = 0.0


In [ ]:
# Initialize the model 
model_1 = model(torch.relu, power, regularization, optimizer='Adam', loss_weights = loss_weights)

In [ ]:
# prepare the data
data_train, data_valid = model_1._prepare_data(0.8, data)

In [ ]:
# Set up the initializing weights and bias
init_weights, init_bias = _sample_uniform_sphere_points(M)

In [ ]:
model_2 = model_outerweights(torch.relu, power, regularization, optimizer='SSN_TR', loss_weights = loss_weights, th = th)

In [ ]:
model_result, weight_raw, bias_raw, outerweight_raw = model_1.train(
    data_train=data_train,
    data_valid=data_valid,
    inner_weights=init_weights, 
    inner_bias=init_bias,
    iterations=1000,
    display_every=200,
)
logger.info("Initialization done"); logger.info(f"Initial weights shape: {weight_raw.shape}, bias shape: {bias_raw.shape}")

In [ ]:
# Training with improved logging
VDP_logger_H1_nc, weight_raw, bias_raw, outerweight_raw = retrain(
    data_train, data_valid, 
    model_1, model_2, model_result, weight_raw, bias_raw, outerweight_raw,
    num_iterations, M, alpha, pruning_threshold, power, gamma
)

logger.info("Training completed with improved logging")

The comparable model is about 126 neurons

In [ ]:
# =============================================================================
# PLOT: Weight space visualization in polar coordinates
# Shows the distribution of weights in 2D space for the BEST model (lowest validation loss)
# =============================================================================

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Extract weights from the current training run
weights_run = VDP_logger_H1_nc.history['weights']
biases_run = VDP_logger_H1_nc.history['biases']
neurons_run = VDP_logger_H1_nc.history['neuron_count']

print(f"Training run: {len(weights_run)} iterations, max neurons: {max(neurons_run)}")

# Use the BEST model (lowest validation loss) instead of the iteration with most neurons
print("Using BEST model (lowest validation loss)")
weights_optimal = VDP_logger_H1_nc.history['best_weights']
b_optimal = VDP_logger_H1_nc.history['best_biases'].reshape(1, -1)   # (1, n)
best_loss = VDP_logger_H1_nc.history['best_test_loss']
print(f"Best validation loss: {best_loss:.6f}")

a_optimal = weights_optimal.T                         # (2, n)
Z = a_optimal / (1 + b_optimal) 

# Create polar coordinate visualization
fig, ax = plt.subplots(1, 1, figsize=(10, 8), subplot_kw={'projection': 'polar'})

# Compute angles and radii in weight space (2D)
angles = np.arctan2(a_optimal[1], a_optimal[0])
r_sphere = np.sqrt(np.sum(a_optimal**2, axis=0) + (b_optimal.flatten())**2)

# Plot in polar coordinates
radius = np.ones_like(angles)
ax.scatter(angles, radius, c=b_optimal.flatten(), cmap='coolwarm', alpha=0.85, s=60)
ax.set_title(f'Weight Space - BEST Model\nNeurons: {weights_optimal.shape[0]}', fontsize=14)
ax.grid(True, alpha=0.5)

# Save the figure
plt.savefig('../data_result/plot/weights_polar_analysis_best.png', dpi=300, bbox_inches='tight')
print(f"Polar coordinate analysis saved to ../data_result/plot/weights_polar_analysis_best.png")

# Show plot
plt.show()

## Test with the L1 Penalty ##

In [ ]:
# Initialize the parameter
power = 2.1
M = 50 # number greedy insertion selected
num_iterations = 10
loss_weights = (1.0, 1.0)
pruning_threshold = 1e-13

gamma = 1e-10
alpha = 1e-5
regularization = (gamma, alpha) 
th = 1.0

The comparable model is about: 207 neurons

In [ ]:
# Initialize the model 
model_1 = model(torch.relu, power, regularization, optimizer='Adam', loss_weights = loss_weights)
model_2 = model_outerweights(torch.relu, power, regularization, optimizer='SSN_TR', loss_weights = loss_weights, th = th)

# Prepare data
data_train, data_valid = model_1._prepare_data(0.8, data)

# Set up the initializing weights and bias
init_weights = np.random.randn(M, 2) * 0.1
init_bias = np.random.randn(M)

model_result, weight_raw, bias_raw, outerweight_raw = model_1.train(
    data_train=data_train,
    data_valid=data_valid,
    inner_weights=init_weights, 
    inner_bias=init_bias,
    iterations=1000,
    display_every=200,
)
logger.info("Initialization done"); logger.info(f"Initial weights shape: {weight_raw.shape}, bias shape: {bias_raw.shape}")

# Training with improved logging
VDP_logger_H1_lasso, weight_raw_1, bias_raw_1, outerweight_raw_1 = retrain(
    data_train, data_valid, model_1, model_2, model_result, weight_raw, bias_raw, outerweight_raw,
    num_iterations, M, alpha, pruning_threshold, power, gamma
)

logger.info("Training completed with improved logging")


In [ ]:
# =============================================================================
# PLOT: Weight space visualization in polar coordinates
# Shows the distribution of weights in 2D space for the BEST model (lowest validation loss)
# =============================================================================

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Extract weights from the current training run
weights_run = VDP_logger_H1_lasso.history['weights']
biases_run = VDP_logger_H1_lasso.history['biases']
neurons_run = VDP_logger_H1_lasso.history['neuron_count']

print(f"Training run: {len(weights_run)} iterations, max neurons: {max(neurons_run)}")

# Use the BEST model (lowest validation loss) instead of the iteration with most neurons
if 'best_weights' in VDP_logger_H1_lasso.history:
    print("Using BEST model (lowest validation loss)")
weights_optimal = VDP_logger_H1_lasso.history['best_weights']
b_optimal = VDP_logger_H1_lasso.history['best_biases'].reshape(1, -1)   # (1, n)
best_loss = VDP_logger_H1_lasso.history['best_test_loss']
print(f"Best validation loss: {best_loss:.6f}")

a_optimal = weights_optimal.T                         # (2, n)
Z = a_optimal / (1 + b_optimal) 

# Create polar coordinate visualization
fig, ax = plt.subplots(1, 1, figsize=(10, 8), subplot_kw={'projection': 'polar'})

# Compute angles and radii in weight space (2D)
angles = np.arctan2(a_optimal[1], a_optimal[0])
r_sphere = np.sqrt(np.sum(a_optimal**2, axis=0) + (b_optimal.flatten())**2)

# Plot in polar coordinates
radius = np.ones_like(angles)
ax.scatter(angles, radius, c=b_optimal.flatten(), cmap='coolwarm', alpha=0.85, s=60)
ax.set_title(f'Weight Space - BEST Model (L1 Penalty)\nNeurons: {weights_optimal.shape[0]}', fontsize=14)
ax.grid(True, alpha=0.5)

# Save the figure
plt.savefig('../data_result/plot/weights_polar_analysis_best_l1.png', dpi=300, bbox_inches='tight')
print(f"Polar coordinate analysis saved to ../data_result/plot/weights_polar_analysis_best_l1.png")

# Show plot
plt.show()


# Optional: Show weight evolution across iterations
if len(weights_run) > 1:
    print(f"\n=== Weight Evolution ===")
    print("Neuron counts across iterations:")
    for i, count in enumerate(neurons_run):
        print(f"Iteration {i}: {count} neurons")